In [1]:
from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torchvision
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [11]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                          shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_of_classes = len(classes)

In [5]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.ConvLayers = nn.Sequential(
            # Input layer 32x32x3
            nn.Conv2d(3, 96, kernel_size=3, padding=1, stride=2),
            nn.ReLU(inplace=True),
            # Convolutional layer 1: (32 - 2 + 2)/2 16x16x96
            nn.MaxPool2d(kernel_size=2),
            # 8x8x96
            nn.Conv2d(96, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # Convolutional layer 2: 8x8x256
            nn.MaxPool2d(kernel_size=2),
            # 4x4x256 
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # Convolutional layer 3: 4x4x384
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # Convolutional layer 4: 4x4x384
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
            # Convolutional layer 4: 2x2x256
        )
        self.FullyConnectedLayers = nn.Sequential(
            # Reducing overfitting
            nn.Dropout(),
            # Fully Connected Layer 1
            nn.Linear(256*2*2, 4096),
            # Fully Connected Layer 2
            nn.Linear(4096, 4096),
            # Output Layer
            nn.Linear(4096, num_of_classes)
        )
        
    def forward(self, x):
        x = self.ConvLayers(x)
        x = x.view(x.size(0), 256*2*2)
        x = self.FullyConnectedLayers(x)
        return x

In [7]:
model = AlexNet()
lr = 0.01
epoch = 5

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
model.to(device)

AlexNet(
  (ConvLayers): Sequential(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (FullyConnectedLayers): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=1024, out_features=4096, bias=True)
    (2): Linear(in_features=4096, out_features=4096, bias=True)
    (3): Linear(in

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [12]:
for e in range(epoch):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = Variable(inputs).to(device), Variable(labels).to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 300 == 299:    # print every 300 mini-batches
            print('[%d, %5d] loss: %f' %
                  (e + 1, i + 1, running_loss))
            running_loss = 0.0

print('Finished Training')

[1,   300] loss: 672.251756
[1,   600] loss: 569.263479
[1,   900] loss: 498.705982
[1,  1200] loss: 461.228710
[1,  1500] loss: 435.688072
[2,   300] loss: 397.056251
[2,   600] loss: 379.518875
[2,   900] loss: 369.782376
[2,  1200] loss: 341.363952
[2,  1500] loss: 329.804908
[3,   300] loss: 301.048505
[3,   600] loss: 292.005040
[3,   900] loss: 281.781141
[3,  1200] loss: 280.591381
[3,  1500] loss: 276.546783
[4,   300] loss: 235.917312
[4,   600] loss: 236.862418
[4,   900] loss: 232.580185
[4,  1200] loss: 239.869304
[4,  1500] loss: 232.295557
[5,   300] loss: 188.345308
[5,   600] loss: 193.551472
[5,   900] loss: 194.987534
[5,  1200] loss: 195.729600
[5,  1500] loss: 200.807163
Finished Training


In [13]:
torch.save(model, 'alexnet.pt')

/home/student/.conda/envs/hientm/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type AlexNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [29]:
temp = model.cpu()

In [33]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = temp(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 70 %


In [34]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = temp(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 78 %
Accuracy of   car : 63 %
Accuracy of  bird : 58 %
Accuracy of   cat : 56 %
Accuracy of  deer : 64 %
Accuracy of   dog : 40 %
Accuracy of  frog : 82 %
Accuracy of horse : 75 %
Accuracy of  ship : 83 %
Accuracy of truck : 89 %
